# Real-time Risk Prediction Demo (Gradio Application)
本ノートブックでは、これまでに構築したBERTモデル、参照用ベクトル、および信頼度算出モデルを統合し、ユーザーがリアルタイムで文章の炎上リスクを判定できるWebアプリケーションを起動します。

## 1. Initialization & Data Preprocessing
推論に必要なライブラリをセットアップし、参照用データセット（Safe/Out）のロードおよびクラス不均衡を是正するためのアンダーサンプリング処理を実施します。

In [ ]:
%pip install deep_translator

In [ ]:
# 1. ライブラリの準備とBERTモデルの読み込み

import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
import random
import matplotlib.pyplot as plt
from deep_translator import GoogleTranslator
import gradio as gr
import os
import pickle

# 事前学習済みBERTモデル (bert-base-uncased) の初期化
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', model_max_length=512)
model = BertModel.from_pretrained('bert-base-uncased')

# 計算リソースの最適化（CUDA環境の優先活用）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 2. キャッシュデータの読み込みとデータセットの整合
# 2.1. 保存済みベクトル、テキスト、およびキャリブレーション・パラメータのロード
with open("bert_vectors_data.pkl", 'rb') as f:
    cache_data = pickle.load(f)
safe_vec_all = cache_data['safe_vec_all']
out_vec_all = cache_data['out_vec_all']

with open("texts_label_data.pkl", 'rb') as f:
    cache_data = pickle.load(f)
safe_txt_all = cache_data['safe_txt_all']
out_txt_all = cache_data['out_txt_all']
out_severe_all = cache_data['out_severe_all']
out_obscene_all = cache_data['out_obscene_all']
out_threat_all = cache_data['out_threat_all']
out_insult_all = cache_data['out_insult_all']
out_identity_all = cache_data['out_identity_all']

# キャリブレーション・モデルのパラメータ取得
SAFE_P = cache_data.get('safe_params')
OUT_P = cache_data.get('out_params')

# 2.2. クラス不均衡の是正（アンダーサンプリング）
random.seed(1) # 再現性確保のためのシード固定
min_size = min(len(safe_vec_all), len(out_vec_all))

# SAFEクラスタのサンプリング
safe_indices = random.sample(range(len(safe_vec_all)), min_size)
safe_vec = [safe_vec_all[i] for i in safe_indices]
safe_txt = [safe_txt_all[i] for i in safe_indices]

# OUTクラスタのサンプリング
out_indices = random.sample(range(len(out_vec_all)), min_size)
out_vec = [out_vec_all[i] for i in out_indices]
out_txt = [out_txt_all[i] for i in out_indices]
out_severe = [out_severe_all[i] for i in out_indices]
out_obscene = [out_obscene_all[i] for i in out_indices]
out_threat = [out_threat_all[i] for i in out_indices]
out_insult = [out_insult_all[i] for i in out_indices]
out_identity = [out_identity_all[i] for i in out_indices]

# 計算用テンソルへの変換
safe_v_all = torch.tensor(np.array(safe_vec)).to(device)
out_v_all = torch.tensor(np.array(out_vec)).to(device)

# 類似度計算の効率化のため、各ベクトルをL2正規化
safe_v_all_norm = torch.nn.functional.normalize(safe_v_all, p=2, dim=1)
out_v_all_norm = torch.nn.functional.normalize(out_v_all, p=2, dim=1)

## 2. Inference Pipeline & UI Logic
ユーザーからの入力を受け取り、翻訳、ベクトル化、類似度計算、および信頼度評価を一連のパイプラインとして実行する推論関数を定義します。また、結果を可視化するためのUIコンポーネントの設定を行います。

In [ ]:
# 1. UIコンポーネントの初期状態定義
# 判定結果のプレースホルダー
initial_res = f"<div style='color: #8899A6; font-size: 48px; font-weight: bold; text-align: center; border: 2px solid #334155; border-radius: 10px; padding: 10px; opacity: 0.6;'>READY</div>"

# 信頼度表示のプレースホルダー
initial_conf = f"<div style='text-align: center; font-size: 24px; color: #8899A6;'>信頼度: <span style='color: #475569; font-size: 32px;'>??.??%</span></div>"

# 2. 分析詳細のテーブル生成関数
def generate_table_html(values=None):
    """
    リスク要因（Toxic/Obscene等）の有無を動的にHTMLテーブル化する。
    """
    labels = ["極めて有害", "わいせつ", "脅迫", "侮辱", "差別的表現"]
    html = "<div style='display: grid; grid-template-columns: repeat(5, 1fr); gap: 1px; margin-top: 0px;'>"
    for i, label in enumerate(labels):
        if values is None: # 待機状態の表示
            symbol, color, bg = "-", "#475569", "rgba(255, 255, 255, 0.02)"
        else:
            val = values[i]
            if val == 0: symbol, color, bg = "○", "#4ade80", "rgba(74, 222, 128, 0.05)"
            elif val == 1: symbol, color, bg = "×", "#f87171", "rgba(248, 113, 113, 0.05)"
            else: symbol, color, bg = "△", "#fbbf24", "rgba(251, 191, 36, 0.05)"

        html += f"""
        <div style='text-align: center; padding: 0px 0px; border-radius: 10px; background: {bg}; border: 1.5px solid {color};'>
            <div style='font-size: 11px; font-weight: bold; color: #8899A6; margin-bottom: 0px; height: 32px; display: flex; align-items: center; justify-content: center;'>{label}</div>
            <div style='font-size: 22px; font-weight: bold; color: {color};'>{symbol}</div>
        </div>"""
    return html + "</div>"

# 3. メイン推論パイプライン (judgement)
def judgement(text_x_jp):
    """
    ユーザー入力を解析し、リスク判定および信頼度スコアを算出する。
    """
    # 入力バリデーション
    if not text_x_jp or text_x_jp.strip() == "":
        return initial_res, initial_conf, "", generate_table_html()

    # 1. 自然言語処理（翻訳とベクトル化）
    # 日本語から英語への自動翻訳
    text_x = GoogleTranslator(source='ja', target='en').translate(text_x_jp)

    model.eval()
    with torch.no_grad():
        # 文章のトークナイズとデバイスへの転送
        inputs = tokenizer(text_x, truncation=True, padding=True, max_length=512, return_tensors='pt').to(device)
        outputs = model(**inputs)

        # BERTの中間層から特徴量を抽出
        token_embeddings = outputs[0]
        input_mask_expanded = inputs['attention_mask'].unsqueeze(-1).expand(token_embeddings.size()).float()

        # 意味のない「埋め合わせ部分（パディング）」を無視するためのマスクを作成
        sentence_vector = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        # コサイン類似度計算用のベクトル正規化
        sentence_vector_norm = torch.nn.functional.normalize(sentence_vector, p=2, dim=1)

    # 2. 参照データとの類似度比較
    # 正規化済みベクトル同士の内積によりコサイン類似度を一括算出
    sim_matrix_safe = torch.mm(sentence_vector_norm, safe_v_all_norm.T)
    sim_matrix_out = torch.mm(sentence_vector_norm, out_v_all_norm.T)

    # 各クラスタにおける最大類似度の特定
    ms_safe, idx_safe = torch.max(sim_matrix_safe.squeeze(0), dim=0)
    ms_out,idx_out = torch.max(sim_matrix_out.squeeze(0), dim=0)

    # 3. クラス判定および信頼度（確率）の算出
    # スコア比較に基づく暫定分類
    if ms_safe > ms_out:
        judge = "safe"
        color = "#00FF7F"
        # 根拠となる類似テキストの抽出と翻訳
        most_similar_text = GoogleTranslator(source='en', target='ja').translate(safe_txt[idx_safe.item()])
        # 属性情報の取得（SAFEのため全項目0）
        severe = 0
        obscene = 0
        threat = 0
        insult = 0
        identity = 0
        # 信頼度算出用の特徴量
        most_similarity_scores = ms_safe
        other_similarity_scores = ms_out
    
    elif ms_safe < ms_out:
        judge = "out"
        color = "#FF4500"
        # 根拠となる類似テキストの抽出と翻訳
        most_similar_text = GoogleTranslator(source='en', target='ja').translate(out_txt[idx_out.item()])
        # 属性情報の取得
        severe = out_severe[idx_out.item()]
        obscene = out_obscene[idx_out.item()]
        threat = out_threat[idx_out.item()]
        insult = out_insult[idx_out.item()]
        identity = out_identity[idx_out.item()]
        # 信頼度算出用の特徴量
        most_similarity_scores = ms_out
        other_similarity_scores = ms_safe
    
    else:
        judge = "error"
        color = "white"
        # 信頼度算出用の特徴量
        most_similarity_scores = 0
        other_similarity_scores = 0
        # 根拠となる類似テキストの抽出と翻訳
        safe_text = GoogleTranslator(source='en', target='ja').translate(safe_txt[idx_safe.item()])
        out_text = GoogleTranslator(source='en', target='ja').translate(out_txt[idx_out.item()])
        most_similar_text = f"{safe_text}\n{out_text}"
        # 属性情報の取得
        if out_severe[idx_out.item()] == 1:
            severe = 3
        else:
            severe = 0
        if out_obscene[idx_out.item()] == 1:
            obscene = 3
        else:
            obscene = 0
        if out_threat[idx_out.item()] == 1:
            threat = 3
        else:
            threat = 0
        if out_insult[idx_out.item()] == 1:
            insult = 3
        else:
            insult = 0
        if out_identity[idx_out.item()] == 1:
            identity = 3
        else:
            identity = 0

    # 4. 信頼度計算
    dif_score = most_similarity_scores - other_similarity_scores
    p = SAFE_P if judge == "safe" else OUT_P
    if p:
        # SAFE用キャリブレーション・モデルの適用 (ロジスティック回帰式)
        logit = p['intercept'] + (p['coef_dif'] * dif_score) + (p['coef_most'] * most_similarity_scores.item())
        prob = 1 / (1 + np.exp(-logit.cpu().numpy()))
        conf_value = f"{prob * 100:.2f}%"
    else:
        conf_value = "---%"

    # 判定表示用HTMLの生成
    if judge == "safe":
        res_html = f"<div style='color: #4ade80; font-size: 48px; font-weight: bold; text-align: center; border: 2px solid #4ade80; border-radius: 10px; padding: 10px;'>SAFE</div>"
        conf_html = f"<div style='text-align: center; font-size: 24px; color: white;'>信頼度: <span style='color: #4ade80; font-size: 32px;'>{conf_value}</span></div>"
    elif judge == "out":
        res_html = f"<div style='color: #f87171; font-size: 48px; font-weight: bold; text-align: center; border: 2px solid #f87171; border-radius: 10px; padding: 10px;'>OUT</div>"
        conf_html = f"<div style='text-align: center; font-size: 24px; color: white;'>信頼度: <span style='color: #f87171; font-size: 32px;'>{conf_value}</span></div>"
    else:
        res_html = "ERROR"
        conf_html = "0%"

    # 5. 属性情報の表示
    risk_vals = [severe, obscene, threat, insult, identity]
    table_html = generate_table_html(risk_vals)

    return res_html, conf_html, most_similar_text, table_html

## 3. Interactive UI Implementation (Gradio Blocks)
`gr.Blocks` レイアウトエンジンを使用して、ダークモード基調のモダンなダッシュボードを構築します。
ユーザー入力、リアルタイム判定結果、統計的信頼度、および判断根拠（類似事例と属性分析）を一つの画面に集約します。

In [ ]:
# Gradio UIコンポーネントの定義とレイアウト構成
with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue", secondary_hue="slate"), css="""
    .gradio-container { background-color: rgba(15, 23, 42, 1.0) !important; }
    .container { max-width: 1000px; margin: auto;}
    .title h1 { text-align: center; margin-bottom: 30px; font-size: 4.5em !important; font-weight: 900; color: #ffffff; letter-spacing: -2px; }
    .equal-height { display: flex !important; flex-direction: column !important; height: 100% !important; padding: 5px; border-radius: 20px; border: 1px solid rgba(255, 255, 255, 0.05); }
    .flex-input { flex-grow: 1 !important; display: flex !important; flex-direction: column !important; background: transparent;}
    .flex-input textarea { border-radius: 12px 12px 12px 12px !important; background: rgba(100,100,100,0.2) !important; color: white !important; border: 1px solid rgba(255,255,255,0.05) !important; }
    .btn-primary { background: #1D9BF0 !important; border: none !important; color: white !important; font-weight: bold !important; height: 65px; font-size: 1.3em !important; border-radius: 12px !important; margin-top: 10px; margin-bottom: 9px; }
    #res_label_area, #res_conf_area, #res_table_area { padding: 0px; border-radius: 12px 12px 12px 12px; border: 0px solid rgba(255,255,255,0.05); border-top: none; background: transparent; }
    .label-only { margin-bottom: -1px !important; background: transparent;}
    .label-only textarea { display: none !important; background: transparent;}
""") as demo:

    with gr.Column(elem_classes="container"):
        gr.Markdown("# 炎上診断AI", elem_classes="title")

        with gr.Row():
            # 左ペイン：入力インターフェース
            with gr.Column(scale=1):
                with gr.Group(elem_classes="equal-height"):
                    input_box = gr.Textbox(
                        label="投稿文入力",
                        placeholder="解析したい文章を入力してください...",
                        lines=18,
                        elem_classes="flex-input",
                    )
                    btn = gr.Button("解析を実行する", variant="primary", elem_classes="btn-primary")

            # 右ペイン：サマリー表示
            with gr.Column(scale=1):
                with gr.Group(elem_classes="equal-height"):

                    # 判定結果セクション
                    gr.Textbox(label="判定結果", interactive=False, elem_classes="label-only")
                    output_label_html = gr.HTML(value=initial_res, elem_id="res_label_area")
                    output_conf_html = gr.HTML(value=initial_conf, elem_id="res_conf_area")
                    output_table_html = gr.HTML(value=generate_table_html(), elem_id="res_table_area")

                    # 過去事例セクション
                    output_text = gr.Textbox(
                        label="最も近い過去の炎上事例",
                        lines=6,
                        interactive=False,
                        elem_classes="flex-input"
                    )

    outputs = [output_label_html, output_conf_html, output_text, output_table_html]
    btn.click(fn=judgement, inputs=[input_box], outputs=outputs)
    input_box.submit(fn=judgement, inputs=[input_box], outputs=outputs)

# サーバーの起動
demo.launch()